# 第一章 `lex`和`yacc`

## `lex`程序

一个典型的lex程序如下所示：

```c
%{
    // header
%}
%%

[\t ]+ ;
is |
am |
go {
        printf("%s: is a verb;\n", yytext);
   }

[a-zA-Z]+ {
              printf("%s: is not a verb", yytext);
          }
%%

int main() {
    yylex();
}
```

* 第一部分：用特殊的界定符号`%{`以及`%}`标记，lex将这一部分的内容直接拷贝到生成的C语言文件中，这一部分可以包含一些后续的C语言代码将会直接使用的部分，例如引用一些后续代码将会引用的头文件，定义一些枚举等等。


* 第二部分：规则段部分，每个规则由两部分组成：
    1. 模式：模式由正则表达式定义。
    2. 动作：当lex识别出某个模式时将执行相应的动作
  
  以上两个部分通过**空格**隔开，多个“规则”对应的“动作”是相同的，那么规则后面可以跟`|`动作，这是一个特殊的动作，意味着和下一个模式使用相同的动作。如果输入的文本能够匹配多个定义的规则，那么lex有一套默认的匹配规则：
    1. lex模式只能匹配输入字符串一次。
    2. lex执行当前输入的最长可能的匹配动作。


* 第三部分：这是用户例程部分，由任意的用户自定义的C语言程序组成，lex将会把用户定义的C语言代码拷贝到最终生成的c代码中。

以上三个部分之间通过特殊符号`%%`来分别标记，在lex程序中，有一些默认的函数和指针可以使用：

* `yytext`：是一个字符指针，当lex匹配到一个模式时，在相应的动作中就可以引用这个指针来指向刚刚匹配到的这个字符串。
* `yylex`：是一个函数，当调用这个函数时lex默认会监听当前的标准输入（即键盘输入），每当输入一行文本按下回车lex程序就会分析刚才输入的文本行。

## `yacc`程序

`yacc`程序是lex程序的更复杂的一层，在词法分析的基础上定义了语法。“语法”可以定义为一组特殊“标记”组成的序列，这些标记即lex分析程序输出，简单来讲可以理解为lex匹配到的“模式”，例如我们可以通过lex分析程序通过定义“模式”的方式来界定英语中的哪些单词是动词，哪些是名词。那么现在如果有一句英文：

```text
I love you;
```

我们可以这样定义lex词法分析程序

```c
%{%}
%%

[\t ]+ ;
I |
you {return NOUN;}

love {return VERB;}

;\n {return 0;}

%%

```

那么当我们调用`yylex()`的时候：lex分析程序在分析输入的`I love you;`时就会分别在检测到相应的单词时返回相应的**token**（即：NOUN，VERB，0）。所谓“语法”就是这些token的有序排列组成的集合，yacc就是为了处理这样的token集合而出现的，在yacc中将这样的token有序集合定义为“语法”，并且针对特定的“语法”可以定义出相应的“动作”（就如同lex分析程序针对不同的模式可以定义出不同的动作一样）

### `yacc`程序的组成部分

一个`yacc`程序和`lex`是相似的，也分为三个部分，第一部分和第三部分和lex分析程序相同，重点是第二部分，即规则的定义部分和lex有所不同，yacc语法分析程序在第二部分进行具体的语法定义：

```c
%{

%}

%token NOUN VERB
%%

sentence: simple_sentence {printf("This is a simple sentence;");}
          | compound_sentence {printf("This is a compound sentence");}
          ;
simple_sentence: NOUN VERB NOUN {printf("This is token sequneces;");}
                ;
compound_sentence: simple_sentence simple_sentence {printf("This is compound;");}
                ;
%%

extern FILE *yyin;
int main() {
    do {
        yyparse();
    } while (!feof(yyin))
}

void yyerror(char *s) {
    fprintf(stderr, "Error: %s", s);
}
```

在第二部分规则定义中，yacc可以定义标记（就像lex一样），所不同的是：

* yacc的标记（即语法）可以是由若干token组成，也可以是由其他的yacc标记组成，或者是其他yacc标记以及token组成的集合
* 一个yacc标记可以由多个“定义”，即一个sentence既可以是一个simple_sentence也可以是一个compound_sentence
* 一个yacc标记可以递归定义

针对lex而言，每个规则只能定义一个动作。但是在yacc中由于一个标记可以有多个“定义”，因此每个“定义”都能定义动作，不同的“定义”之间使用特殊符号`|`连接，最后以`;`结尾。

### `yacc`的内置函数

yacc和lex一样也拥有内置函数，在本例中就是yyparse，这个函数需要调用lex分析程序中的yylex来首先进行词法分析，获取输入文本中的token集合，根据这些token集合再去匹配相应的语法规则。在yacc的定义文件中可以通过`%token`这样的特殊写法来定义token，在lex文件中通过`#include "y.tab.h"`来感知yacc定义文件中的这些token。

在实际的分析过程中，lex需要告诉yacc“什么时候一句话结尾了”。换句话说，如果现在有很多英文句子：
```text
I love you; you love me; I play games;
```
yacc其实并不知道什么时候断句，然后将一整句话进行语法分析，这个“断句”的信息其实是lex给出的，lex会通过返回一个特殊的token（即0）告诉yacc程序当前的句子已经结束，可以根据之前返回的token序列进行语法分析了。

举一个具体的例子，假设现在有三句话（每句话以分号和换行结尾）：

```text
I love you;
you love me;
```

此时调用`yyparse`，函数内部会先调用yylex进行语法分析，yylex首先读到I，然后返回NOUN，然后依次返回VERB和NOUN，最后读取到`;\n`返回0告诉yacc程序当前句子已经结束，yacc就会根据lex之前返回的`NOUN VERB NOUN`进行语法匹配，匹配到simple_sentence这个语法规则，然后制定语法文件中定义的动作，此时yyparse也会返回，此次调用就结束。

### yacc与lex的通信

根据上面的表述可以知道，yacc需要和lex配合才能够正常工作，毕竟yacc的语法规则本身就是lex标记的序列集合。它们之间的通信依靠的是`y.tab.h`这个头文件完成的，这个头文件是yacc自动生成的，通过在lex中引用这个头文件可以引用到yacc中定义的一些内容，例如`%token`定义。